In [2]:
from __future__ import print_function
from builtins import input

import os.path
import tempfile
import pyfftw   # See https://github.com/pyFFTW/pyFFTW/issues/40
import numpy as np
import scipy.io as sio

from sporco import util
from sporco import signal
from sporco import plot
plot.config_notebook_plotting()
from sporco.metric import psnr
from sporco.cupy import (cupy_enabled, np2cp, cp2np, select_device_by_load,
                         gpu_info)
from sporco.cupy.admm import pdcsc
from sporco.dictlrn import bpdndl

In [3]:
def pad(x, n=8):

    if x.ndim == 2:
        return np.pad(x, n, mode='symmetric')
    else:
        return np.pad(x, ((n, n), (n, n), (0, 0)), mode='symmetric')


def crop(x, n=8):

    return x[n:-n, n:-n]

In [4]:
img = util.ExampleImages().image('a.jpg', zoom=0.5, scaled=True,
                                 idxexp=np.s_[:, :])
np.random.seed(12345)
imgn = signal.spnoise(img, 0.44)

In [5]:
D0 = util.convdicts()['G:8x8x32']
Di = np.zeros(D0.shape[0:2] + (1,), dtype=np.float32)
Di[0, 0] = 1.0
D = np.concatenate((Di, D0), axis=2)

S = img.reshape((-1, img.shape[-1])).T
np.random.seed(12345)
B0 = np.random.randn(S.shape[0], 20)
lmbda = 0.02
opt = bpdndl.BPDNDictLearn.Options({'Verbose': True, 'MaxMainIter': 100,
                                    'BPDN': {'rho': 10.0*lmbda + 0.1},
                                    'CMOD': {'rho': S.shape[1] / 2e2}})

d = bpdndl.BPDNDictLearn(B0, S, lmbda, opt)
B = d.solve()

Itn   Fnc       DFid      ℓ1        Cnstr     r_X       s_X       ρ_X       r_D       s_D       ρ_D     
--------------------------------------------------------------------------------------------------------
   0  2.29e+03  1.92e+03  1.87e+04  8.67e-01  5.21e-01  6.82e-01  3.00e-01  1.98e-01  5.37e-01  2.98e+02
   1  1.99e+03  1.16e+03  4.13e+04  1.11e+00  3.25e-01  9.79e-01  3.00e-01  2.47e-01  8.41e-01  2.98e+02
   2  9.15e+02  4.54e+02  2.31e+04  9.05e-01  3.01e-01  7.14e-01  3.00e-01  2.08e-01  4.98e-01  2.98e+02
   3  7.09e+02  2.08e+02  2.51e+04  6.09e-01  2.81e-01  1.69e-01  3.00e-01  1.26e-01  2.90e-01  2.98e+02
   4  7.52e+02  1.28e+02  3.12e+04  5.25e-01  1.75e-01  2.84e-01  3.00e-01  1.18e-01  2.29e-01  2.98e+02
   5  6.15e+02  1.19e+02  2.48e+04  4.07e-01  1.36e-01  2.66e-01  3.00e-01  8.67e-02  8.89e-02  2.98e+02
   6  5.74e+02  5.31e+01  2.60e+04  2.77e-01  1.10e-01  9.36e-02  3.00e-01  6.20e-02  6.36e-02  2.98e+02
   7  5.81e+02  1.45e+01  2.83e+04  2.38e-01  7.50e-02 

  77  5.45e+02  1.21e+01  2.67e+04  8.44e-04  1.95e-03  5.29e-03  3.00e-01  5.95e-04  3.81e-03  1.49e+02
  78  5.45e+02  1.20e+01  2.67e+04  8.43e-04  1.92e-03  5.21e-03  3.00e-01  5.91e-04  3.78e-03  1.49e+02
  79  5.45e+02  1.20e+01  2.67e+04  8.09e-04  1.89e-03  5.14e-03  3.00e-01  5.85e-04  3.75e-03  1.49e+02
  80  5.45e+02  1.20e+01  2.67e+04  7.75e-04  1.86e-03  5.07e-03  3.00e-01  5.80e-04  3.74e-03  1.49e+02
  81  5.45e+02  1.20e+01  2.67e+04  7.50e-04  1.83e-03  5.00e-03  3.00e-01  5.76e-04  3.72e-03  1.49e+02
  82  5.45e+02  1.20e+01  2.67e+04  7.42e-04  1.81e-03  4.94e-03  3.00e-01  5.72e-04  3.70e-03  1.49e+02
  83  5.45e+02  1.20e+01  2.67e+04  7.29e-04  1.78e-03  4.88e-03  3.00e-01  5.67e-04  3.67e-03  1.49e+02
  84  5.45e+02  1.20e+01  2.67e+04  7.19e-04  1.75e-03  4.82e-03  3.00e-01  5.62e-04  3.64e-03  1.49e+02
  85  5.45e+02  1.20e+01  2.67e+04  7.06e-04  1.73e-03  4.76e-03  3.00e-01  5.58e-04  3.62e-03  1.49e+02
  86  5.45e+02  1.20e+01  2.67e+04  6.86e-04  1.71e-03 

In [6]:
lmbda = 1.4e0
mu = 9e0

In [7]:
wl1 = np.ones((1,)*4 + (D.shape[2],), dtype=np.float32)
wl1[..., 0] = 0.0

In [8]:
wgr = np.zeros((D.shape[2]), dtype=np.float32)
wgr[0] = 1.0

In [9]:
opt = pdcsc.ConvProdDictL1L1Grd.Options(
    {'Verbose': True, 'MaxMainIter': 100, 'RelStopTol': 5e-3,
     'AuxVarObj': False, 'rho': 1e1, 'RelaxParam': 1.8,
     'L1Weight': np2cp(wl1), 'GradWeight': np2cp(wgr)})

In [ ]:
if not cupy_enabled():
    print('CuPy/GPU device not available: running without GPU acceleration\n')
else:
    id = select_device_by_load()
    info = gpu_info()
    if info:
        print('Running on GPU %d (%s)\n' % (id, info[id].name))

b = pdcsc.ConvProdDictL1L1Grd(np2cp(D), np2cp(B), np2cp(pad(imgn)),
                              lmbda, mu, opt=opt, dimK=0)
X = cp2np(b.solve())

CuPy/GPU device not available: running without GPU acceleration

Itn   Fnc       DFid      Regℓ1     Regℓ2∇    r         s       
----------------------------------------------------------------
   0  1.53e+05  7.99e+03  1.04e+05  2.52e+01  2.06e-01  2.19e+00
   1  1.39e+05  1.61e+04  8.66e+04  1.72e+02  1.52e-01  2.85e+00


In [ ]:
imgdp = cp2np(b.reconstruct().squeeze())
imgd = crop(imgdp)


In [ ]:
print("ConvProdDictL1L1Grd solve time: %5.2f s" % b.timer.elapsed('solve'))
print("Noisy image PSNR:    %5.2f dB" % psnr(img, imgn))
print("Denoised image PSNR: %5.2f dB" % psnr(img, imgd))

In [ ]:
fig, ax = plot.subplots(nrows=1, ncols=3, figsize=(21, 7))
fig.suptitle('ConvProdDictL1L1GrdJoint Results (false colour, '
             'bands 10, 20, 30)')
plot.imview(img, title='Reference', ax=ax[0], fig=fig)
plot.imview(imgn, title='Noisy', ax=ax[1], fig=fig)
plot.imview(imgd, title='Denoised', ax=ax[2], fig=fig)
fig.show()

In [ ]:
its = b.getitstat()
ObjFun = [float(x) for x in its.ObjFun]
PrimalRsdl = [float(x) for x in its.PrimalRsdl]
DualRsdl = [float(x) for x in its.DualRsdl]
fig = plot.figure(figsize=(20, 5))
plot.subplot(1, 3, 1)
plot.plot(ObjFun, xlbl='Iterations', ylbl='Functional', fig=fig)
plot.subplot(1, 3, 2)
plot.plot(np.vstack((PrimalRsdl, DualRsdl)).T,
          ptyp='semilogy', xlbl='Iterations', ylbl='Residual',
          lgnd=['Primal', 'Dual'], fig=fig)
plot.subplot(1, 3, 3)
plot.plot(its.Rho, xlbl='Iterations', ylbl='Penalty Parameter', fig=fig)
fig.show()